In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer
import folium

# ======================================
# 1. Load WFD River, Canal, and Surface Water Transfer Waterbodies
# ======================================

wfd = gpd.read_file("/content/WFD_River_Canal_and_Surface_Water_Transfer_Water_Bodies_Cycle_2.geojson")

# Simplify geometry for performance (optional)
wfd["geometry"] = wfd["geometry"].simplify(tolerance=0.001)

# ======================================
# 2. Load Data Centres
# ======================================
# Example CSV format:
# postcode,lon,lat
# SW1A1AA,-0.141,51.501

data = pd.read_csv("/content/combined_postcodes_geocoded.csv")

# Make into GeoDataFrame
gdf_data = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.lon, data.lat),
    crs="EPSG:4326"
)

# ======================================
# 3. Create the base map
# ======================================
m = folium.Map(location=[54.5, -3.0], zoom_start=6, tiles="CartoDB positron")

# ======================================
# 4. Create Feature Groups (toggleable)
# ======================================
layer_water = folium.FeatureGroup(name="Rivers, Canals & Surface Water", show=True)
layer_data = folium.FeatureGroup(name="Data Centres", show=True)

# ======================================
# 5. Add Waterbody Layer
# ======================================
folium.GeoJson(
    wfd,
    name="WFD Waterbodies",
    style_function=lambda feature: {
        "color": "blue",
        "weight": 1,
        "opacity": 0.6,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["wb_name", "wb_cat"],
        aliases=["Name", "Type"]
    )
).add_to(layer_water)

# ======================================
# 6. Add Data Centres Layer
# ======================================
for _, row in gdf_data.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=4,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.8,
        popup=f"<b>Data Centre</b><br>{row.get('postcode', 'Unknown')}"
    ).add_to(layer_data)

# ======================================
# 7. Add layers to map + legend
# ======================================
layer_water.add_to(m)
layer_data.add_to(m)

legend_html = """
<div style="
position: fixed;
bottom: 20px; left: 20px; width: 160px; height: 100px;
background-color: white; z-index:9999; font-size:14px;
border:2px solid grey; border-radius:10px; padding:10px;">
<b>Legend</b><br>
<i style='color:blue'>━</i> Waterbodies<br>
<i style='color:red'>⬤</i> Data Centres
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# ======================================
# 8. Add controls and save
# ======================================
folium.LayerControl(collapsed=False).add_to(m)
m.save("uk_waterbodies_datacentres_map.html")

print("✅ Map saved as 'uk_waterbodies_datacentres_map.html'")


✅ Map saved as 'uk_waterbodies_datacentres_map.html'
